In [1]:
import cv2
import numpy as np
import time
import os

In [2]:
labelsPath = os.path.sep.join(['./', "obj.names"])
LABELS = open(labelsPath).read().strip().split("\n")
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
    dtype="uint8")


In [3]:
# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join(["./weights/", "yolov4-obj_best.weights"])
configPath = os.path.sep.join(["./models", "yolov4-obj.cfg"])

In [4]:
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

[INFO] loading YOLO from disk...


In [5]:
args = {"confidence":0.5, "threshold":0.3}

In [6]:
imagepath_input = '/home/andrei/scripts/yolo-object-detection/images'

for f in os.listdir(imagepath_input):
    if f.endswith('.jpg'):
        image_path = os.path.join(imagepath_input, f)
        image = cv2.imread(image_path)
        (H, W) = image.shape[:2]
        ln = net.getLayerNames()
        ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
        blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
        net.setInput(blob)
        start = time.time()
        layerOutputs = net.forward(ln)
        end = time.time()
        print("[INFO] YOLO took {:.6f} seconds".format(end - start))
        # initialize our lists of detected bounding boxes, confidences, and
        # class IDs, respectively
        boxes = []
        confidences = []
        classIDs = []

        # loop over each of the layer outputs
        for output in layerOutputs:
            # loop over each of the detections
            for detection in output:
                # extract the class ID and confidence (i.e., probability) of
                # the current object detection
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]

                # filter out weak predictions by ensuring the detected
                # probability is greater than the minimum probability
                if confidence > args["confidence"]:
                    # scale the bounding box coordinates back relative to the
                    # size of the image, keeping in mind that YOLO actually
                    # returns the center (x, y)-coordinates of the bounding
                    # box followed by the boxes' width and height
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")

                    # use the center (x, y)-coordinates to derive the top and
                    # and left corner of the bounding box
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))

                    # update our list of bounding box coordinates, confidences,
                    # and class IDs
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

        # apply non-maxima suppression to suppress weak, overlapping bounding
        # boxes
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, args["confidence"],
            args["threshold"])

        # ensure at least one detection exists
        if len(idxs) > 0:
            classCounts = dict()
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                classID = classIDs[i]
                label = LABELS[classID]
                if (classID not in classCounts):
                    classCounts[classID] = 0
                classCounts[classID] = classCounts[classID] + 1

                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])

                # draw a bounding box rectangle and label on the image
                color = [int(c) for c in COLORS[classID]]
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                
                text = "{}: {:.2f}%".format(label, 100*confidences[i])
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)
            
            for index, classID in enumerate(classCounts):
                label = LABELS[classID]
                count = classCounts[classID] 
                text = "{0}: {1}".format(label, count) 
                print(text)
                cv2.putText(image, text, (20, 20 + index*16), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, [0,0,0], 2)
        # show the output image
        imageOutputPath = os.path.join('output', image_path.split('/')[len(image_path.split('/'))-1])
        print(imageOutputPath)
        cv2.imwrite(imageOutputPath, image)
        # cv2.imshow("Image", image)
        # cv2.waitKey(0)
        print('\n')        


[INFO] YOLO took 1.105548 seconds
ambulance: 1
output/1.jpg


[INFO] YOLO took 0.860637 seconds
ambulance: 1
output/52.9.jpg


[INFO] YOLO took 0.813184 seconds
ambulance: 2
output/55.9.jpg


[INFO] YOLO took 0.722325 seconds
ambulance: 1
output/68.9.jpg


[INFO] YOLO took 0.714034 seconds
ambulance: 1
output/71.7.jpg


[INFO] YOLO took 0.748138 seconds
ambulance: 1
output/70.7.jpg


[INFO] YOLO took 0.737509 seconds
ambulance: 1
output/56.9.jpg


[INFO] YOLO took 0.719933 seconds
ambulance: 1
output/69.7.jpg


[INFO] YOLO took 0.725865 seconds
ambulance: 1
output/65.9.jpg


[INFO] YOLO took 0.740478 seconds
ambulance: 1
output/54.7.jpg


[INFO] YOLO took 0.730498 seconds
ambulance: 2
output/66.9.jpg


[INFO] YOLO took 0.737412 seconds
ambulance: 1
output/70.9.jpg


[INFO] YOLO took 0.706654 seconds
ambulance: 2
output/50.9.jpg


[INFO] YOLO took 0.754708 seconds
ambulance: 1
output/66.7.jpg


[INFO] YOLO took 0.773674 seconds
ambulance: 1
output/49.9.jpg


[INFO] YOLO took 0.721818 se